# Data Cleaning Practice  

Goals
- Practice data cleaning with Python
- Seek concise code
- Clearly document decisions
- Be thorough 

Datasets:
- Top grossing tours data scraped from wikipedia
    - Small dataset
- Something else
    - Large dataset
    - Seek computationally efficient ways to clean the data
        - Dask?

In [24]:
### Packages
import numpy as np
# import pandas as pd
import modin.pandas as pd # faster
import os
from janitor import clean_names
os.chdir('C:/Users/WulfN/')

### Read in Data

# Simple wikipedia dataset
messy = (clean_names(
    pd.read_csv('./datasets/unclean_data_practice/wikipedia_scraped.csv'))
    .rename(columns={'adjusted_gross_in_2022_dollars_': 'adjusted_gross_in_2022_dollars', 
                     'year_s_': 'year_range'}))

pd.options.display.float_format = '{:20,.2f}'.format

In [25]:
messy

,rank,peak,all_time_peak,actual_gross,adjusted_gross_in_2022_dollars,artist,tour_title,year_range,shows,average_gross,ref_
0,1,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1]
1,2,1,7[2],"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3]
2,3,1[4],2[5],"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6]
3,4,2[7],10[7],"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018–2019,156,"$2,546,795",[7]
4,5,2[4],NaN,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8]
5,6,2[4],10[9],"$305,158,363","$388,978,496",Madonna,The MDNA Tour,2012,88,"$3,467,709",[9]
6,7,2[10],NaN,"$280,000,000","$381,932,682",Celine Dion,Taking Chances World Tour,2008–2009,131,"$2,137,405",[11]
7,7,NaN,NaN,"$257,600,000","$257,600,000",Pink,Summer Carnival †,2023–2024,41,"$6,282,927",[12]
8,9,NaN,NaN,"$256,084,556","$312,258,401",Beyoncé,The Formation World Tour,2016,49,"$5,226,215",[13]
9,10,NaN,NaN,"$250,400,000","$309,141,878",Taylor Swift,The 1989 World Tour,2015,85,"$2,945,882",[14]


The wikipedia page the data is scraped from is not specificied, though it appears to be top grossing tours. 

Here are questions and areas for improvements:
- Make column names lower case and seperate words by an '_'
- 2 rows are ranked 7, why?
- remove notation like [1]
- The gross columns of ',' and '$' that need to be removed
- Tour title has unnecessary symbols (only keep letters, numbers, and spaces)
- Years could be seperated into start_year and end_year
- data types of aforementioned variables needs to be changed from 'object' to integer in most cases
- perhaps an artist level table could be made showing stats by artist

In [26]:
messy.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   rank                            20 non-null     int64 
 1   peak                            9 non-null      object
 2   all_time_peak                   6 non-null      object
 3   actual_gross                    20 non-null     object
 4   adjusted_gross_in_2022_dollars  20 non-null     object
 5   artist                          20 non-null     object
 6   tour_title                      20 non-null     object
 7   year_range                      20 non-null     object
 8   shows                           20 non-null     int64 
 9   average_gross                   20 non-null     object
 10  ref_                            20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


### Data Cleaning

In [29]:
# Data cleaning

# Split messy strings and clean integers
messy_int = messy.select_dtypes(include='int64')
messy_strings = messy.select_dtypes(include='object')

# Fix rank value in messy_int
messy_int.at[7, 'rank'] = 8

# Clean messy strings
messy_strings = (messy_strings
                 # correct column names 
                 .rename(columns={'adjusted_gross_in_2022_dollars_': 'adjusted_gross_in_2022_dollars', 
                                  'year_s_': 'year_range'})
                 # split year_range into 'year_start' and 'year_end'
                 .assign(year_start = lambda x: x['year_range'].str[:4],
                         # *** Assuming year_end = year_start if no year_end provided ***
                         year_end = lambda x: x['year_range'].str[-4:]) 
                 .drop(columns=['ref_', 'year_range'])
                 # remove notation like [1]
                 .replace(to_replace=r'\[.*', value='', regex = True)
                 # remove ALL non number or letter symbols (aside from é)
                 .replace(to_replace=r'[^a-zA-Z0-9 é]', value='', regex=True) 
)

In [34]:
### Preserve column order, remove ref_ column, and include updated columns
column_order = list(messy.columns)
column_order.remove('ref_') 

column_loc = column_order.index('year_range')
column_order[column_loc:column_loc + 1] = ['year_start', 'year_end'] # .replace(['year_range'], ['year_start', 'year_end'])

### Merge and restore order of columns
clean = pd.concat([messy_int, messy_strings], axis=1) # on index level
clean = clean[column_order]

In [32]:
### Create dictionary with datatypes and update dataframe

# Partition columns into lists
string_cols = ['artist', 'tour_title']
date_cols = ['year_start', 'year_end']

convert_to_float = list(clean.columns)

for col in string_cols + date_cols:
    convert_to_float.remove(col) 

# create dictionary
data_type_colnames = convert_to_float + date_cols + string_cols
datatypes = ['float'] * len(convert_to_float) + ['int'] * len(date_cols) + ['str'] * len(string_cols)

datatype_conversion = dict(zip(data_type_colnames, datatypes))

# convert
clean = clean.astype(datatype_conversion)
clean


,rank,shows,peak,all_time_peak,actual_gross,adjusted_gross_in_2022_dollars,artist,tour_title,average_gross,year_start,year_end
0,1.00,56.00,1.00,2.00,"780,000,000.00","780,000,000.00",Taylor Swift,The Eras Tour,"13,928,571.00",2023,2024
1,2.00,56.00,1.00,7.00,"579,800,000.00","579,800,000.00",Beyoncé,Renaissance World Tour,"10,353,571.00",2023,2023
2,3.00,85.00,1.00,2.00,"411,000,000.00","560,622,615.00",Madonna,Sticky Sweet Tour,"4,835,294.00",2008,2009
3,4.00,156.00,2.00,10.00,"397,300,000.00","454,751,555.00",Pink,Beautiful Trauma World Tour,"2,546,795.00",2018,2019
4,5.00,53.00,2.00,NaN,"345,675,146.00","402,844,849.00",Taylor Swift,Reputation Stadium Tour,"6,522,173.00",2018,2018
5,6.00,88.00,2.00,10.00,"305,158,363.00","388,978,496.00",Madonna,The MDNA Tour,"3,467,709.00",2012,2012
6,7.00,131.00,2.00,NaN,"280,000,000.00","381,932,682.00",Celine Dion,Taking Chances World Tour,"2,137,405.00",2008,2009
7,8.00,41.00,NaN,NaN,"257,600,000.00","257,600,000.00",Pink,Summer Carnival,"6,282,927.00",2023,2024
8,9.00,49.00,NaN,NaN,"256,084,556.00","312,258,401.00",Beyoncé,The Formation World Tour,"5,226,215.00",2016,2016
9,10.00,85.00,NaN,NaN,"250,400,000.00","309,141,878.00",Taylor Swift,The 1989 World Tour,"2,945,882.00",2015,2015


### Simple Transformations / Feature Engineering

- Create artist level dataframe with the following stats
    - Give qty of tours by artist
    - Average ranking on the list
    - Average # shows

In [33]:
artist_stats = (clean.groupby('artist')
                .agg(qty_tours = ('artist', 'count'),
                     avg_rank = ('rank', 'mean'),
                     avg_show_num = ('shows', 'mean'),
                     adjusted_gross_per_tour = ('adjusted_gross_in_2022_dollars', 'mean'))
                .sort_values('avg_rank', ascending=True))
artist_stats

,qty_tours,avg_rank,avg_show_num,adjusted_gross_per_tour
artist,,,,
Celine Dion,1,7.00,131.00,"381,932,682.00"
Beyoncé,3,7.33,79.00,"391,753,765.67"
Taylor Swift,4,9.00,70.00,"419,352,459.00"
Pink,3,9.33,113.00,"313,267,967.33"
Madonna,4,10.50,78.75,"359,566,225.25"
Katy Perry,1,13.00,151.00,"251,856,802.00"
Cher,1,14.00,325.00,"299,676,265.00"
Lady Gaga,2,14.50,150.50,"254,435,025.00"
Adele,1,19.00,121.00,"204,486,106.00"
